In [ ]:
!pip install optuna

X_tmp, y_tmp = train_feature, train_target # each is a numpy array

X = pd.DataFrame(X_tmp)
y = pd.DataFrame(y_tmp)

from sklearn.metrics import mean_absolute_error as MAE

def objective(trial):
    MAE_cv = []
  # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    params_lgb = {
        "criterion" : trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 300, 3000),
        "objective": "regression",
        "metric": "MAE",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 150),
        "max_bin": trial.suggest_int("max_bin", 200, 600)
    }
  

    kf = KFold(n_splits = 5)
    kf.get_n_splits(X)

    for train_index, valid_index in kf.split(X):
        
    # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        model = LGBMRegressor(**params_lgb)
        model.fit(
          X_train,
          y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)], # early stopping에 쓰일
          early_stopping_rounds=100,
          verbose= False,
      )

        y_pred = model.predict(X_valid)
        MAE_cv.append( MAE(y_valid, y_pred) )
        #MAE = cross_val_score(model, X_train, y_valid, scoring="neg_mean_absolute_error", n_jobs = -1, cv = 5).mean()
    
    MAE_cv = np.mean(MAE_cv)
    return MAE_cv



sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=100)
print("Best score :", study.best_value)
print("Best params combination :", study.best_trial.params)